In [0]:
from pyspark.sql.functions import regexp_extract, regexp_replace, trim, col, when

# Load the existing invoice table
blank_df = spark.table("databricks_catalog.invoice_schema.raw_inovice_table")

# Remove Invoice with Blank Total
cleansed_df = (
    blank_df.na.drop()
    .withColumn("total_clean", trim(regexp_replace(col("total"), "[$,]", "")).cast("double"))
    .filter(col("total_clean") != 0)
    .drop("total_clean")
)

display(cleansed_df)

# Save the cleaned data to Delta
cleansed_df.write.format("delta").mode("overwrite").saveAsTable(
    "databricks_catalog.invoice_schema.cleansed_invoice_table"
)